In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv("C:/datasets/research_task/data_train.csv")

In [3]:
data_train

,id,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,...,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,resolved_by,target
0,0,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,Created by 6,...,?,True,False,Do Not Notify,?,?,?,?,Resolved by 149,130
1,1,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,Created by 171,...,Resolver 89,True,False,Do Not Notify,?,?,?,?,Resolved by 81,149
2,2,True,0,0,4,True,Caller 4416,Opened by 8,29/2/2016 06:10,?,...,?,True,False,Do Not Notify,Problem ID 2,?,?,?,Resolved by 5,140
3,3,True,0,0,0,True,Caller 4491,Opened by 180,29/2/2016 06:38,Created by 81,...,Resolver 125,True,False,Do Not Notify,?,?,?,?,Resolved by 113,174
4,4,True,1,0,5,True,Caller 3765,Opened by 180,29/2/2016 06:58,Created by 81,...,?,True,False,Do Not Notify,?,?,?,?,Resolved by 62,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10567,10567,True,0,0,0,True,Caller 272,Opened by 47,28/4/2016 20:28,?,...,?,False,False,Do Not Notify,?,?,?,?,Resolved by 24,120
10568,10568,True,0,0,0,True,Caller 4020,Opened by 58,28/4/2016 21:07,Created by 31,...,Resolver 17,False,False,Do Not Notify,?,?,?,?,Resolved by 15,132
10569,10569,True,1,0,3,True,Caller 2175,Opened by 305,28/4/2016 23:04,Created by 130,...,Resolver 115,False,False,Do Not Notify,?,?,?,?,Resolved by 103,232
10570,10570,True,1,0,5,True,Caller 2988,Opened by 463,28/4/2016 23:29,Created by 202,...,Resolver 249,False,False,Do Not Notify,?,?,?,?,Resolved by 227,250


In [4]:
data_train = data_train.replace('?', np.nan)

In [5]:
data_train.describe()

,id,reassignment_count,reopen_count,sys_mod_count,target
count,10572.000000,10572.000000,10572.0,10572.000000,10572.000000
mean,5285.500000,0.719921,0.0,1.668842,341.463678
std,3052.017857,1.148499,0.0,2.892447,451.432343
min,0.000000,0.000000,0.0,0.000000,0.000000
25%,2642.750000,0.000000,0.0,0.000000,125.000000
50%,5285.500000,0.000000,0.0,1.000000,213.000000
75%,7928.250000,1.000000,0.0,2.000000,402.000000
max,10571.000000,13.000000,0.0,56.000000,5380.000000


In [6]:
data_v = data_train[data_train['target'] > 720].index

In [7]:
data_train = data_train.drop(data_v, axis=0)
data_train = data_train.drop(["id", "reopen_count"], axis=1)

In [8]:
data_train.describe()

,reassignment_count,sys_mod_count,target
count,9906.000000,9906.000000,9906.000000
mean,0.699172,1.498890,252.862104
std,1.106884,2.198124,146.944348
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,124.000000
50%,0.000000,1.000000,194.000000
75%,1.000000,2.000000,360.000000
max,12.000000,56.000000,719.000000


In [9]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9906 entries, 0 to 10571
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   active                   9906 non-null   bool  
 1   reassignment_count       9906 non-null   int64 
 2   sys_mod_count            9906 non-null   int64 
 3   made_sla                 9906 non-null   bool  
 4   caller_id                9904 non-null   object
 5   opened_by                9440 non-null   object
 6   opened_at                9906 non-null   object
 7   sys_created_by           6384 non-null   object
 8   sys_created_at           6384 non-null   object
 9   sys_updated_by           9906 non-null   object
 10  sys_updated_at           9906 non-null   object
 11  contact_type             9906 non-null   object
 12  location                 9861 non-null   object
 13  category                 9888 non-null   object
 14  subcategory              9858 non-null 

In [10]:
data_train = data_train.dropna(axis = 1, thresh=1000)

In [11]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9906 entries, 0 to 10571
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   active                   9906 non-null   bool  
 1   reassignment_count       9906 non-null   int64 
 2   sys_mod_count            9906 non-null   int64 
 3   made_sla                 9906 non-null   bool  
 4   caller_id                9904 non-null   object
 5   opened_by                9440 non-null   object
 6   opened_at                9906 non-null   object
 7   sys_created_by           6384 non-null   object
 8   sys_created_at           6384 non-null   object
 9   sys_updated_by           9906 non-null   object
 10  sys_updated_at           9906 non-null   object
 11  contact_type             9906 non-null   object
 12  location                 9861 non-null   object
 13  category                 9888 non-null   object
 14  subcategory              9858 non-null 

In [12]:
pd.unique(data_train["sys_created_at"])

array(['29/2/2016 01:23', '29/2/2016 04:57', nan, ..., '29/4/2016 00:04',
       '28/4/2016 23:56', '28/4/2016 23:57'], dtype=object)

In [13]:
trash_data = ["active", "made_sla"]

In [14]:
data_train = data_train.drop(trash_data, axis=1)

In [15]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9906 entries, 0 to 10571
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   reassignment_count       9906 non-null   int64 
 1   sys_mod_count            9906 non-null   int64 
 2   caller_id                9904 non-null   object
 3   opened_by                9440 non-null   object
 4   opened_at                9906 non-null   object
 5   sys_created_by           6384 non-null   object
 6   sys_created_at           6384 non-null   object
 7   sys_updated_by           9906 non-null   object
 8   sys_updated_at           9906 non-null   object
 9   contact_type             9906 non-null   object
 10  location                 9861 non-null   object
 11  category                 9888 non-null   object
 12  subcategory              9858 non-null   object
 13  u_symptom                7475 non-null   object
 14  impact                   9906 non-null 

In [16]:
id_cols = ["caller_id", "sys_created_by", "sys_updated_by", "location", "assigned_to", "assignment_group",
           "resolved_by", "opened_by", "category", "subcategory", "u_symptom"]
data_cols = ["opened_at", "sys_created_at", "sys_updated_at"]

n_id_cols = ["caller_id", "sys_created_by", "sys_updated_by", "assigned_to", "assignment_group",
           "resolved_by", "opened_by"]

In [17]:
data_train["data_opened_at"] = data_train["opened_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_train["data_sys_created_at"] = data_train["sys_created_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_train["data_sys_updated_at"] = data_train["sys_updated_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_train["data_created_updated_diff"] = (data_train["sys_updated_at"].astype("datetime64") - 
    data_train["sys_created_at"].astype("datetime64")).astype("timedelta64[h]")
data_train["data_opened_created_diff"] = (data_train["sys_created_at"].astype("datetime64") - 
    data_train["opened_at"].astype("datetime64")).astype("timedelta64[h]")



#data_train = data_train.drop(n_id_cols, axis=1)
data_train = data_train.drop(data_cols, axis=1)

In [18]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9906 entries, 0 to 10571
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   reassignment_count         9906 non-null   int64  
 1   sys_mod_count              9906 non-null   int64  
 2   caller_id                  9904 non-null   object 
 3   opened_by                  9440 non-null   object 
 4   sys_created_by             6384 non-null   object 
 5   sys_updated_by             9906 non-null   object 
 6   contact_type               9906 non-null   object 
 7   location                   9861 non-null   object 
 8   category                   9888 non-null   object 
 9   subcategory                9858 non-null   object 
 10  u_symptom                  7475 non-null   object 
 11  impact                     9906 non-null   object 
 12  urgency                    9906 non-null   object 
 13  priority                   9906 non-null   obje

In [19]:
data_train.describe()

,reassignment_count,sys_mod_count,target,data_created_updated_diff,data_opened_created_diff
count,9906.000000,9906.000000,9906.000000,6384.000000,6384.000000
mean,0.699172,1.498890,252.862104,84.381736,0.124687
std,1.106884,2.198124,146.944348,1013.670576,9.012609
min,0.000000,0.000000,0.000000,-6341.000000,-3.000000
25%,0.000000,0.000000,124.000000,0.000000,0.000000
50%,0.000000,1.000000,194.000000,0.000000,0.000000
75%,1.000000,2.000000,360.000000,7.000000,0.000000
max,12.000000,56.000000,719.000000,7322.000000,720.000000


In [20]:
data_n_cu = data_train[data_train['data_created_updated_diff'] < 0].index
data_v_cu = data_train[data_train['data_created_updated_diff'] > 1000].index
data_train = data_train.drop(data_v_cu, axis=0)
data_train = data_train.drop(data_n_cu, axis=0)
data_n_oc = data_train[data_train['data_opened_created_diff'] < 0].index
data_train = data_train.drop(data_n_oc, axis=0)

In [21]:
data_train["data_created_updated_diff"] = data_train["data_created_updated_diff"].replace(np.nan, 
                                                                                          data_train["data_created_updated_diff"].mean())
data_train["data_opened_created_diff"] = data_train["data_opened_created_diff"].replace(np.nan, 
                                                                                          data_train["data_opened_created_diff"].mean())

In [22]:
data_train.describe()

,reassignment_count,sys_mod_count,target,data_created_updated_diff,data_opened_created_diff
count,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000
mean,0.611404,1.311781,244.766003,54.341417,0.137537
std,0.984952,2.020086,143.187689,137.740065,7.468640
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,123.000000,0.000000,0.000000
50%,0.000000,1.000000,184.000000,9.000000,0.000000
75%,1.000000,2.000000,339.000000,54.341417,0.137537
max,12.000000,56.000000,719.000000,951.000000,720.000000


In [23]:
data_train = data_train.replace(np.nan, '?')

In [24]:
cat_cols = [col for col in data_train.select_dtypes(include=['object']).columns]
num_cols = [col for col in data_train.select_dtypes(include=['int64', 'bool', 'float64']).columns]

In [25]:
dummy_features = pd.get_dummies(data_train[cat_cols])

In [26]:
data_train = pd.concat([data_train[num_cols], dummy_features], axis=1)

In [27]:
f_cols = [col for col in data_train.columns if (col != 'target')]
f_num_cols = [col for col in num_cols if (col != 'target')]

In [28]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9295 entries, 0 to 10571
Columns: 5117 entries, reassignment_count to data_sys_updated_at_9-m
dtypes: bool(2), float64(2), int64(3), uint8(5110)
memory usage: 45.7 MB


In [29]:
data_train = data_train.astype('float')

In [30]:
#Min-Max нормировка
data_train["data_created_updated_diff"] = (data_train["data_created_updated_diff"] - data_train["data_created_updated_diff"].min()) / (data_train["data_created_updated_diff"].max() - data_train["data_created_updated_diff"].min())
data_train["data_opened_created_diff"] = (data_train["data_opened_created_diff"] - data_train["data_opened_created_diff"].min()) / (data_train["data_opened_created_diff"].max() - data_train["data_opened_created_diff"].min())
data_train["sys_mod_count"] = (data_train["sys_mod_count"] - data_train["sys_mod_count"].min()) / (data_train["sys_mod_count"].max() - data_train["sys_mod_count"].min())
data_train["reassignment_count"] = (data_train["reassignment_count"] - data_train["reassignment_count"].min()) / (data_train["reassignment_count"].max() - data_train["reassignment_count"].min())

In [31]:
data_train.describe()

,reassignment_count,sys_mod_count,knowledge,u_priority_confirmation,target,data_created_updated_diff,data_opened_created_diff,caller_id_?,caller_id_Caller 10,caller_id_Caller 1000,...,data_sys_updated_at_11-m,data_sys_updated_at_12-m,data_sys_updated_at_2-m,data_sys_updated_at_3-m,data_sys_updated_at_4-m,data_sys_updated_at_5-m,data_sys_updated_at_6-m,data_sys_updated_at_7-m,data_sys_updated_at_8-m,data_sys_updated_at_9-m
count,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,...,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000,9295.000000
mean,0.050950,0.023425,0.208069,0.000538,244.766003,0.057141,0.000191,0.000215,0.000108,0.000108,...,0.049381,0.028402,0.037977,0.305003,0.342657,0.035395,0.025175,0.050672,0.047983,0.018935
std,0.082079,0.036073,0.405948,0.023188,143.187689,0.144837,0.010373,0.014668,0.010372,0.010372,...,0.216675,0.166128,0.191152,0.460433,0.474624,0.184787,0.156664,0.219340,0.213741,0.136303
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,123.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.017857,0.000000,0.000000,184.000000,0.009464,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.083333,0.035714,0.000000,0.000000,339.000000,0.057141,0.000191,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,719.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_train[f_cols], data_train['target'], 
                                                       train_size=0.8, 
                                                       random_state=42)

In [33]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
import random
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset
from torchvision import datasets

In [34]:
class MyDataset(Dataset):
    def __init__(self, samples, labels):
        self.labels = torch.from_numpy(labels.to_numpy()).type(torch.FloatTensor) 
        self.samples = torch.from_numpy(samples.to_numpy()).type(torch.FloatTensor) 

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        label = self.labels[idx]
        return sample, label

In [35]:
t_data = MyDataset(X_train, y_train)
v_data = MyDataset(X_test, y_test)

In [36]:
dataloaders = {'train': data.DataLoader(t_data, batch_size=1,
                                              shuffle=True, num_workers=0),
               'val': data.DataLoader(v_data, batch_size=1,
                                              shuffle=True, num_workers=0)
               }
dataset_sizes = {'train': len(t_data),
                'val': len(v_data)}

In [37]:
from tqdm.autonotebook import tqdm, trange

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time

In [38]:
use_gpu = torch.cuda.is_available()

In [39]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_loss = 20000
    
    losses = {'train': [], "val": []}

    pbar = trange(num_epochs, desc="Epoch:")

    for epoch in pbar:
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.eval()

            running_loss = 0.0

            for data in tqdm(dataloaders[phase], leave=False, desc=f"{phase} iter:"):
                inputs, labels = data
                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                #labels = labels.to(torch.float32)
                if phase=="train":
                    optimizer.zero_grad()

                # forward pass
                if phase == "eval":
                    with torch.no_grad():
                        outputs = model(inputs)
                else:
                    outputs = model(inputs)
                preds = torch.argmax(outputs, -1)
                loss = criterion(outputs, labels)

                # backward pass + оптимизируем только если это стадия обучения
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # статистика
                running_loss += loss.item()

            epoch_loss = running_loss / dataset_sizes[phase]
            
            losses[phase].append(epoch_loss)
            
            pbar.set_description('{} Loss: {:.4f}'.format(
                                    phase, epoch_loss
                                ))

            # если достиглось лучшее качество, то запомним веса модели
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # загрузим лучшие веса модели
    model.load_state_dict(best_model_wts)
    return model, losses

In [40]:
def evaluate(model):
    model.eval()
    
    runninig_correct = 0
    for data in dataloaders['val']:
        inputs, labels = data
        
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
            
        output = model(inputs)
        _, predicted = torch.max(output, 1)
        
        runninig_correct += int(torch.sum(predicted == labels))
        
    return runninig_correct / dataset_sizes['val']

In [53]:
class SimpleNet(nn.Module):
    def __init__(self, num):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(num, 500)
        self.dropout1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(500, 500)
        #self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(500, 1)
        
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = self.dropout1(F.sigmoid(self.fc2(x)))
        x = self.fc3(x)
        return x

In [56]:
model = SimpleNet(len(f_cols))
if use_gpu:
    model = model.cuda()
loss_fn = nn.L1Loss()
optimizer_ft = optim.Adam(model.parameters(), lr=1e-3)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [57]:
model, losses = train_model(model, loss_fn, optimizer_ft, num_epochs=10)

Epoch::   0%|          | 0/10 [00:00<?, ?it/s]

train iter::   0%|          | 0/7436 [00:00<?, ?it/s]

val iter::   0%|          | 0/1859 [00:00<?, ?it/s]

train iter::   0%|          | 0/7436 [00:00<?, ?it/s]

val iter::   0%|          | 0/1859 [00:00<?, ?it/s]

...

train iter::   0%|          | 0/7436 [00:00<?, ?it/s]

val iter::   0%|          | 0/1859 [00:00<?, ?it/s]

train iter::   0%|          | 0/7436 [00:00<?, ?it/s]

val iter::   0%|          | 0/1859 [00:00<?, ?it/s]

Training complete in 3m 46s
Best val Loss: 69.396988
